In [78]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
from sqlalchemy import create_engine
import backoff

In [2]:
from config import SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET, user_info, connection_string

In [3]:
#Set up Spotify credentials
client_credentials_manager = SpotifyClientCredentials(SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [34]:
# Set up genre csv
genres_df = pd.read_csv("../../Cleaning Codes/genre_artist.csv")
genres_df.head()

,id,followers,genres,name,popularity
0,0VLMVnVbJyJ4oyZs2L3Yl2,71.0,carnaval cadiz,Las Viudas De Los Bisabuelos,6
1,0dt23bs4w8zx154C5xdVyl,63.0,carnaval cadiz,Los De Capuchinos,5
2,0pGhoB99qpEJEsBQxgaskQ,64.0,carnaval cadiz,Los “Pofesionales”,7
3,3HDrX2OtSuXLW5dLR85uN3,53.0,carnaval cadiz,Los Que No Paran De Rajar,6
4,22mLrN5fkppmuUPsHx6i2G,59.0,"classical harp, harp",Vera Dulova,3


In [35]:
# rename columns for later join
genres_df.drop(columns = ['id', 'followers', 'popularity'], inplace = True)
genres_df.rename(columns = {'name':'artist'}, inplace = True)

In [22]:
#Set up Hot 100 csv
hot100 = pd.read_csv('charts.csv')

# Any empty Hot 100 cells?
hot100.isnull().sum()

date                  0
rank                  0
song                  0
artist                0
last-week         31971
peak-rank             0
weeks-on-board        0
dtype: int64

In [23]:
# Clean DataFrame
hot100 = hot100.drop(columns = ['last-week', 'peak-rank'])
hot100.head()

,date,rank,song,artist,weeks-on-board
0,2021-06-12,1,Butter,BTS,2
1,2021-06-12,2,Good 4 U,Olivia Rodrigo,3
2,2021-06-12,3,Levitating,Dua Lipa Featuring DaBaby,35
3,2021-06-12,4,Leave The Door Open,Silk Sonic (Bruno Mars & Anderson .Paak),13
4,2021-06-12,5,Save Your Tears,The Weeknd & Ariana Grande,25


In [24]:
hot100.dtypes

date              object
rank               int64
song              object
artist            object
weeks-on-board     int64
dtype: object

In [25]:
# Transform date into a date-time value
hot100["date"] = pd.to_datetime(hot100["date"])
hot100.dtypes

date              datetime64[ns]
rank                       int64
song                      object
artist                    object
weeks-on-board             int64
dtype: object

In [26]:
# Sort by date
hot100 = hot100.sort_values(by = 'date', ascending = False)
hot100.head()

,date,rank,song,artist,weeks-on-board
0,2021-06-12,1,Butter,BTS,2
64,2021-06-12,65,amari,J. Cole,3
75,2021-06-12,76,What's Next,Drake,13
73,2021-06-12,74,4 Da Gang,42 Dugg & Roddy Ricch,9
72,2021-06-12,73,Quicksand,Morray,17


In [79]:
# Get the desired timeframe
selecthot100 = hot100.loc[(hot100['date'] >= '2010-01-01') & (hot100['date'] <='2020-01-31')]

In [80]:
def getTrackFeatures(song_id):
    #Query Spotify
    meta = sp.track(song_id)
    features = sp.audio_features(song_id)
    
    track= {
        # Meta data
        "song": meta['name'],
        "artist": meta['album']['artists'][0]['name'],
        "release_date": meta['album']['release_date'],
        "popularity": meta['popularity'],

        # features
        "acousticness": features[0]['acousticness'],
        "danceability": features[0]['danceability'],
        "energy": features[0]['energy'],
        "instrumentalness": features[0]['instrumentalness'],
        "liveness": features[0]['liveness'],
        "loudness": features[0]['loudness'],
        "speechiness": features[0]['speechiness'],
        "tempo": features[0]['tempo']
    }
    
    return track

In [90]:
def searchSongs(title, artist):
    # Clean artist for Spotify query
    cleaned_artist = artist.replace(" Featuring", "")
    hot_song_features = {}
    
    # Query database, get top result and call getTrackFeatures to get audio features for track
    try:
        results = sp.search(q=f'track:{title} artist:{cleaned_artist}')
        top_result = results['tracks']['items'][0]
        hot_song_features = getTrackFeatures(top_result['id'])
    except:
        
        #If query fails, search without artist
        try:
            results = sp.search(q=f'track:{title}')
            top_result = results['tracks']['items'][0]
            hot_song_features = getTrackFeatures(top_result['id'])
        except:
            # Print songs that fail.
            print(f'Failed to find {title} by {artist}.')
    return hot_song_features

In [84]:
def getSongData(row):
    # API calls automatically rate limited by Spotipy
    song_details = searchSongs(row['song'], row['artist'])
    
    #Make dictionary based on data from searchSongs
    song_details['popular_date'] = row['date']
    song_details['ranking'] = row['rank']
    song_details['weeks_on_board'] = row['weeks-on-board']
    return song_details

In [91]:
# Get data from Spotify using custom functions
hot_song_features = [getSongData(song) for idx, song in selecthot100.iterrows()]

Failed to find Don't Check On Me by Chris Brown Featuring Justin Bieber & Ink.
Failed to find Can't Leave Without It by 21 Savage.
Failed to find Can't Leave Without It by 21 Savage.
Failed to find Lost In Japan by Shawn Mendes X Zedd.
Failed to find Lost In Japan by Shawn Mendes X Zedd.
Failed to find Can't Leave Without It by 21 Savage.
Failed to find Can't Leave Without It by 21 Savage.
Failed to find Lost In Japan by Shawn Mendes X Zedd.
Failed to find Lost In Japan by Shawn Mendes X Zedd.
Failed to find Lost In Japan by Shawn Mendes X Zedd.
Failed to find Lost In Japan by Shawn Mendes X Zedd.
Failed to find Lost In Japan by Shawn Mendes X Zedd.
Failed to find Lost In Japan by Shawn Mendes X Zedd.
Failed to find Lost In Japan by Shawn Mendes X Zedd.
Failed to find Lost In Japan by Shawn Mendes X Zedd.
Failed to find Lost In Japan by Shawn Mendes X Zedd.
Failed to find Lost In Japan by Shawn Mendes X Zedd.
Failed to find Lost In Japan by Shawn Mendes X Zedd.
Failed to find Lost In J

In [92]:
# Make DataFrame with Spotify data
cleaned_hot_songs = pd.DataFrame(data = hot_song_features)
cleaned_hot_songs

,song,artist,release_date,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,popular_date,ranking,weeks_on_board
0,Take What You Want (feat. Ozzy Osbourne & Trav...,Post Malone,2019-09-06,76.0,0.09060,0.499,0.800,0.000000,0.1470,-2.665,0.0502,139.919,2020-01-25,70,19
1,RITMO (Bad Boys For Life),Black Eyed Peas,2019-10-12,78.0,0.03340,0.721,0.716,0.000840,0.2370,-7.037,0.0657,104.994,2020-01-25,67,8
2,BEST ON EARTH (feat. BIA) - Bonus,Russ,2020-01-31,71.0,0.18800,0.846,0.748,0.000000,0.0930,-3.512,0.0780,90.006,2020-01-25,68,6
3,Tusa,Various Artists,2019-12-20,45.0,0.29500,0.803,0.715,0.000134,0.0574,-3.280,0.2980,101.085,2020-01-25,69,7
4,I Wish Grandpas Never Died,Riley Green,2019-09-20,71.0,0.03600,0.445,0.492,0.000000,0.1220,-5.717,0.0294,148.013,2020-01-25,75,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52595,Evacuate The Dancefloor - Radio Edit,Cascada,2009,60.0,0.01950,0.760,0.696,0.000000,0.2780,-5.811,0.0491,127.029,2010-01-02,28,19
52596,Live Like We're Dying,Kris Allen,2009-11-16,52.0,0.02730,0.589,0.893,0.000000,0.3430,-2.948,0.0397,92.011,2010-01-02,33,8
52597,Say Aah (feat. Fabolous),Trey Songz,2009-08-31,63.0,0.00453,0.724,0.870,0.000000,0.8330,-3.614,0.1130,93.010,2010-01-02,34,6
52598,Gangsta Luv,Snoop Dogg,2009-01-01,53.0,0.02770,0.590,0.735,0.000000,0.0818,-4.924,0.2970,102.076,2010-01-02,35,9


In [93]:
# add in genre
genre_songs = pd.merge(cleaned_hot_songs, genres_df, how = 'left', on = ['artist'])
genre_songs

,song,artist,release_date,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,popular_date,ranking,weeks_on_board,genres
0,Take What You Want (feat. Ozzy Osbourne & Trav...,Post Malone,2019-09-06,76.0,0.09060,0.499,0.800,0.000000,0.1470,-2.665,0.0502,139.919,2020-01-25,70,19,"dfw rap, melodic rap, rap"
1,RITMO (Bad Boys For Life),Black Eyed Peas,2019-10-12,78.0,0.03340,0.721,0.716,0.000840,0.2370,-7.037,0.0657,104.994,2020-01-25,67,8,"dance pop, pop, pop rap"
2,BEST ON EARTH (feat. BIA) - Bonus,Russ,2020-01-31,71.0,0.18800,0.846,0.748,0.000000,0.0930,-3.512,0.0780,90.006,2020-01-25,68,6,"hawaiian hip hop, pop rap, rap"
3,BEST ON EARTH (feat. BIA) - Bonus,Russ,2020-01-31,71.0,0.18800,0.846,0.748,0.000000,0.0930,-3.512,0.0780,90.006,2020-01-25,68,6,uk drill
4,Tusa,Various Artists,2019-12-20,45.0,0.29500,0.803,0.715,0.000134,0.0574,-3.280,0.2980,101.085,2020-01-25,69,7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53078,Evacuate The Dancefloor - Radio Edit,Cascada,2009,60.0,0.01950,0.760,0.696,0.000000,0.2780,-5.811,0.0491,127.029,2010-01-02,28,19,"bubblegum dance, dance pop, eurodance, europop..."
53079,Live Like We're Dying,Kris Allen,2009-11-16,52.0,0.02730,0.589,0.893,0.000000,0.3430,-2.948,0.0397,92.011,2010-01-02,33,8,"acoustic pop, idol, indiecoustica, neo mellow,..."
53080,Say Aah (feat. Fabolous),Trey Songz,2009-08-31,63.0,0.00453,0.724,0.870,0.000000,0.8330,-3.614,0.1130,93.010,2010-01-02,34,6,"dance pop, pop, post-teen pop, r&b, trap, urba..."
53081,Gangsta Luv,Snoop Dogg,2009-01-01,53.0,0.02770,0.590,0.735,0.000000,0.0818,-4.924,0.2970,102.076,2010-01-02,35,9,"g funk, gangster rap, hip hop, pop rap, rap, w..."


In [99]:
# Make sure data fits Postgres Primary Key requirements
cleaned_genre_songs = genre_songs.drop_duplicates(subset = ['song', 'popular_date'])
cleaned_genre_songs = 

,song,artist,release_date,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,popular_date,ranking,weeks_on_board,genres
0,Take What You Want (feat. Ozzy Osbourne & Trav...,Post Malone,2019-09-06,76.0,0.0906,0.499,0.800,0.000000,0.1470,-2.665,0.0502,139.919,2020-01-25,70,19,"dfw rap, melodic rap, rap"
1,RITMO (Bad Boys For Life),Black Eyed Peas,2019-10-12,78.0,0.0334,0.721,0.716,0.000840,0.2370,-7.037,0.0657,104.994,2020-01-25,67,8,"dance pop, pop, pop rap"
2,BEST ON EARTH (feat. BIA) - Bonus,Russ,2020-01-31,71.0,0.1880,0.846,0.748,0.000000,0.0930,-3.512,0.0780,90.006,2020-01-25,68,6,"hawaiian hip hop, pop rap, rap"
4,Tusa,Various Artists,2019-12-20,45.0,0.2950,0.803,0.715,0.000134,0.0574,-3.280,0.2980,101.085,2020-01-25,69,7,NaN
5,I Wish Grandpas Never Died,Riley Green,2019-09-20,71.0,0.0360,0.445,0.492,0.000000,0.1220,-5.717,0.0294,148.013,2020-01-25,75,3,"contemporary country, country road"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Dance Monkey,Tones And I,2019-10-17,84.0,0.6920,0.824,0.588,0.000104,0.1490,-6.400,0.0924,98.027,2020-01-25,8,15,"australian pop, pop"
68,"10,000 Hours (with Justin Bieber)",Dan + Shay,2019-10-04,83.0,0.1530,0.654,0.630,0.000000,0.1110,-4.644,0.0259,89.991,2020-01-25,7,15,"contemporary country, country, pop"
69,Someone You Loved,Lewis Capaldi,2019-05-17,87.0,0.7510,0.501,0.405,0.000000,0.1050,-5.679,0.0319,109.891,2020-01-25,6,36,"pop, uk pop"
70,Lose You To Love Me,Selena Gomez,2020-01-10,81.0,0.5560,0.488,0.343,0.000000,0.2100,-8.985,0.0436,102.819,2020-01-25,5,13,"dance pop, pop, post-teen pop"


In [95]:
# Connect to Postgres
db_string = f'postgresql://{user_info}@{connection_string}'
engine = create_engine(db_string)

In [106]:
# Send to Postgres
cleaned_genre_songs.to_sql('popular_songs', con = engine, if_exists = 'append', index = False, chunksize=5200)